# Agent框架与策略分析

## 其他Agent认知框架

### Plan-and-Execute

计划与执行（Plan-and-Execute）框架侧重于先规划一系列的行动，然后执行。这个框架可以使大模型能够先综合考虑任务的多个方面，然后按照计划进行行动。应用在比较复杂的项目管理中或者需要多步决策的场景下会比较合适。

<img src="./img/plan-and-execute.png">

In [2]:
from langchain_openai import ChatOpenAI
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain import SerpAPIWrapper
from langchain.agents.tools import Tool
from langchain import LLMMathChain
from dotenv import load_dotenv
load_dotenv()
# 配置API密钥和基础URL
llm = ChatOpenAI()

# https://serpapi.com/dashboard
# 创建工具
search = SerpAPIWrapper() 
llm_math_chain = LLMMathChain(llm=llm, verbose=True)

# 定义工具列表
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="用于回答关于当前事件的问题"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="用于计算或解决问题"
    )
]

# 加载规划器和执行器
planner = load_chat_planner(llm)
executor = load_agent_executor(llm, tools, verbose=True)

# 创建Plan and Execute代理
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)

# 运行代理解决实际问题
print(agent.run("在纽约，100美元能买几束玫瑰？"))


/Users/yuejunzhang/anaconda3/envs/AI_LLM/lib/python3.11/site-packages/langchain/chains/llm_math/base.py:57: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(




> Entering new PlanAndExecute chain...
steps=[Step(value='确定玫瑰的价格：查找纽约地区的玫瑰花价格。'), Step(value='确定玫瑰的束数：计算100美元可以购买多少束玫瑰花。'), Step(value="给出回答：将计算结果回答给用户。\nGiven the above steps taken, please respond to the user's original question.\n")]

> Entering new AgentExecutor chain...
Thought: I need to search for the current objective to find the price of roses in the New York area.
Action:
```
{
  "action": "Search",
  "action_input": "Find the price of roses in New York area"
}
```
Observation: ['We have the best roses that you will find. When you combine our quality and dependability, you get the very best for a great value!', 'Celebrate any occasion with beautiful and fresh roses, hand-delivered in NYC and NJ by PlantShed, your local Manhattan florist.', 'Ravishing Dozen Rose Arrangement in New York, NY | FLOWERS BY RICHARD NYC Ravishing Dozen Rose Arrangement Shown at $100.00 Buy Now', 'Pink Roses Gift Box - Long Stemmed Roses · From $85 · $95 ; Wildflowers · From $89 · $99 ; Sunflowers 

### Self-Ask

自问自答（Self-Ask）框架这个允许大模型对自己提出问题并回答，来增强对问题的理解以提高回答质量，这个框架在需要深入分析或者提供创造性解决方案下可以比较适合，例如创意写作。

<img src="./img/Self-Ask.webp">

In [ ]:

import os
from langchain import hub
from langchain.agents import AgentExecutor, create_self_ask_with_search_agent
from langchain_community.tools.tavily_search import TavilyAnswer
from dotenv import load_dotenv
load_dotenv()
# 配置模型
from langchain_fireworks import Fireworks

llm = Fireworks(
    # https://fireworks.ai/account/billing
    api_key=os.getenv("FIREWORKS_API_KEY"),
    # model="accounts/fireworks/models/mixtral-8x7b-instruct",
    model="accounts/fireworks/models/llama-v3p1-405b-instruct",
    max_tokens=256)

tools = [TavilyAnswer(max_results=1, name="Intermediate Answer",tavily_api_key=os.getenv("TAVILY_API_KEY"))]
#获取prompt
prompt = hub.pull("hwchase17/self-ask-with-search")
print(prompt)
agent = create_self_ask_with_search_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True,handle_parsing_errors=True)
print(agent_executor.invoke({"input": "上一届的美国总统是谁?"}))

### Thinking and Self-Refection

思考并自我反思（Thinking and Self-Refection）框架主要用于模拟和实现复杂决策过程，通过不断自我评估和调整，使系统能够学习并改进决策过程，从而在面对复杂问题是作出更加有效的决策。

**langgraph**  
**rankrag**

<img src="./img/Thinking and Self-Reflection.webp">

## ReAct框架回顾

<img src="./img/react回顾.webp">

在langchain使用Agent中，我们重点需要理解下面4个元素。
+ 1、llm: 提供逻辑的引擎，负责生成预测和处理输入。
+ 2、prompt: 负责指导模型，形成推理框架
+ 3、tools: 外部工具的使用 包含数据增强、清洗、搜索引擎、api等等。
+ 4、Agent Executor: 负责调用合适的外部工具，并管理整个流程。


In [5]:
# 导入环境变量
from dotenv import load_dotenv
load_dotenv()

# 初始化大模型
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4-turbo-preview',
             temperature=0.5)

# 设置工具
from langchain_community.agent_toolkits.load_tools import load_tools
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# 设置提示模板
from langchain.prompts import PromptTemplate
template = ('''
    '尽你所能用中文回答以下问题。如果能力不够你可以使用以下工具:\n\n'
    '{tools}\n\n
    Use the following format:\n\n'
    'Question: the input question you must answer\n'
    'Thought: you should always think about what to do\n'
    'Action: the action to take, should be one of [{tool_names}]\n'
    'Action Input: the input to the action\n'
    'Observation: the result of the action\n'
    '... (this Thought/Action/Action Input/Observation can repeat N times)\n'
    'Thought: I now know the final answer\n'
    'Final Answer: the final answer to the original input question\n\n'
    'Begin!\n\n'
    'Question: {input}\n'
    'Thought:{agent_scratchpad}'
    '''
)
prompt = PromptTemplate.from_template(template)

# 初始化Agent
from langchain.agents import create_react_agent
agent = create_react_agent(llm, tools, prompt)

# 构建AgentExecutor
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent,
                               tools=tools,
                               handle_parsing_errors=True,
                               verbose=True)

# 执行AgentExecutor
agent_executor.invoke({"input": """目前市场上玫瑰花的一般进货价格是多少？\n如果我在此基础上加价5%，应该如何定价？"""})






> Entering new AgentExecutor chain...
我需要先找到目前市场上玫瑰花的一般进货价格，然后在此基础上加价5%来计算最终的售价。

Action: Search
Action Input: 目前市场上玫瑰花的一般进货价格['以玫瑰为例，一扎（20枝）产自云南的普通C级玫瑰花往年价格为20至30元，今年最高价已涨至200至300元不等，价格翻了10倍。寓意长长久久的999朵玫瑰花束售价 ...', '元旦来临，云南鲜切花市场迎来销售高峰期，价格飙升，卡罗拉红玫瑰平时10-20元一把，现在能卖到40-50元。涨价潮可能持续到春节。 “昨天拍一支花可能 ...', '今年由于气温回升较快，玫瑰花已经开始采摘。平阴县玫瑰鲜花蕾开秤价格从2.8元/斤，迅速攀升至4月27日的3.5元/斤，开秤价格平均每斤比去年同期上涨1 ...', "在记者走访了解中，不少商户直呼“今年七夕节玫瑰花价格是近几年来最低的一次”。“红玫瑰一捆20支的价格一般在25-40元，像'弗洛伊德'这类特定花色的精品玫瑰 ...", '我公司每天会按照我们网站上公布的鲜花报价发货,如有我们网站上公布的价格与实际的发货价格不符，请与我们网站上的客服联系或者直接致电给我们询问. 鲜花等级说明：. A级 ...', '一位花店负责人表示，因为价格高，买到的花材有限，店里今年只备了两束99朵红玫瑰的花束，售价2066元。“玫瑰几乎是花店界的硬通货了，订花材的时候，实在 ...', '“去年情人节我们单支玫瑰包装后卖15元，今年可能要卖到20元。” 据了解，现在批发市场1扎红玫瑰每天至少涨价10-15元，虽然不少人已经望而却步，但商家均表 ...', '周先生说，由于临近情人节，玫瑰花的销量在逐天上涨，价格也比平时翻了好几倍。“拿红玫瑰举例，平时这个品种一把进价在30元左右，一把20支，现在已经涨到 ...', '店家告诉记者，这是专门为今年兔年打造的情人节花束，共有11支玫瑰，此类包装精致的玫瑰花花束价格都在200元左右，包装费在50-60元左右。常见的用包装纸 ...', '4天前首页>> 520朵红玫瑰价格:低高喜好:低高× 520朵玫瑰,.. 平时价:3899元七夕节价:4864元520朵玫瑰,.. 平时价:4000元七夕节价:4992元520朵 ...']从搜索结果来看，市

{'input': '目前市场上玫瑰花的一般进货价格是多少？\n如果我在此基础上加价5%，应该如何定价？',
 'output': '如果以32.5元作为玫瑰花的平均进货价格，在此基础上加价5%，最终的售价应该是34.13元（四舍五入到小数点后两位）。'}

## AgentExecutor运行机制

##### 具体上课进行源码分析

In [ ]:
base.py agent.py 
on_chain_start
self._call
_take_next_step

## 通过llamindex实现ReAct RAG Agent

In [4]:
# 加载电商财报数据
from llama_index.core import SimpleDirectoryReader
from dotenv import load_dotenv
load_dotenv()
A_docs = SimpleDirectoryReader(
    input_files=["./A.pdf"]
).load_data()
B_docs = SimpleDirectoryReader(
    input_files=["./B.pdf"]
).load_data()



# 从文档中创建索引
from llama_index.core import VectorStoreIndex
A_index = VectorStoreIndex.from_documents(A_docs)
B_index = VectorStoreIndex.from_documents(B_docs)

# 持久化索引（保存到本地）
from llama_index.core import StorageContext
A_index.storage_context.persist(persist_dir="./storage/A")
B_index.storage_context.persist(persist_dir="./storage/B")


# 从本地读取索引
from llama_index.core import load_index_from_storage
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/A"
    )
    A_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/B"
    )
    B_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False


# 创建查询引擎
A_engine = A_index.as_query_engine(similarity_top_k=3)
B_engine = B_index.as_query_engine(similarity_top_k=3)


# 配置查询工具
from llama_index.core.tools import QueryEngineTool
from llama_index.core.tools import ToolMetadata
query_engine_tools = [
    QueryEngineTool(
        query_engine=A_engine,
        metadata=ToolMetadata(
            name="A_Finance",
            description=(
                "用于提供A公司的财务信息 "
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=B_engine,
        metadata=ToolMetadata(
            name="B_Finance",
            description=(
                "用于提供A公司的财务信息 "
            ),
        ),
    ),
]


# 配置大模型
from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-4")


# 创建ReAct Agent
from llama_index.core.agent import ReActAgent
agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)


# 让Agent完成任务
print(agent.chat("Compare the sales of the two companies"))


Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 21 0 (offset 0)
Ignoring wrong pointing object 23 0 (offset 0)
Ignoring wrong pointing object 28 0 (offset 0)
Ignoring wrong pointing object 33 0 (offset 0)
Ignoring wrong pointing object 41 0 (offset 0)
Ignoring wrong pointing object 43 0 (offset 0)
Ignoring wrong pointing object 52 0 (offset 0)
Ignoring wrong pointing object 54 0 (offset 0)
Ignoring wrong pointing object 56 0 (offset 0)
Ignoring wrong pointing object 58 0 (offset 0)
Ignoring wrong pointing object 60 0 (offset 0)
Ignoring wrong pointing object 66 0 (offset 0)
Ignoring wrong pointing object 71 0 (offset 0)
Ignoring wrong pointing object 73 0 (offset 0)
Ignoring wrong pointing object 78 0 (offset 0)
Ignoring wrong pointing object 80 0 (offset 0)
Ignoring wrong pointing object 85 0 (offset 0)
Ignoring wrong 

> Running step f31bcb07-c5c0-43a2-95b9-8b4e75a1d9e8. Step input: Compare the sales of the two companies
Thought: The current language of the user is English. I need to use the A_Finance tool to get the sales information of company A first.
Action: A_Finance
Action Input: {'input': 'sales'}
Observation: Sales and marketing expenses increased by 12.4% to US$ 918.0 million in the third quarter of 2023 from US$ 816.7 million in the third quarter of 2022. Core marketplace revenue, which includes transaction-based fees and advertising revenues, was up 31.7% year-on-year to US$ 1.3 billion.
> Running step 2df10410-2566-44e4-8004-2cd08fddd0c3. Step input: None
Thought: I have obtained the sales information for company A. Now I need to use the B_Finance tool to get the sales information for company B.
Action: B_Finance
Action Input: {'input': 'sales'}
Observation: The sales figures for the Taobao and Tmall Group, Alibaba International Digital Commerce Group, Local Services Group, Cainiao Smart 